In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Import data
data_path = '/kaggle/input/machine-predictive-maintenance-classification/predictive_maintenance.csv'
data = pd.read_csv(data_path)
n = data.shape[0]
# First checks
data.info()
print('Check for duplicate values:',data['Product ID'].unique().shape[0]!=n)

In [ ]:
# Set numeric columns dtype to float
data['Tool wear [min]'] = data['Tool wear [min]'].astype('float64')
data['Rotational speed [rpm]'] = data['Rotational speed [rpm]'].astype('float64')
# Rename features
data.rename(mapper={'Air temperature [K]': 'Air temperature',
                    'Process temperature [K]': 'Process temperature',
                    'Rotational speed [rpm]': 'Rotational speed',
                    'Torque [Nm]': 'Torque',
                    'Tool wear [min]': 'Tool wear'}, axis=1, inplace=True)

In [ ]:
# Remove first character and set to numeric dtype
data['Product ID'] = data['Product ID'].apply(lambda x: x[1:])
data['Product ID'] = pd.to_numeric(data['Product ID'])

In [ ]:
# Drop ID columns
df = data.copy()
df.drop(columns=['UDI','Product ID'], inplace=True)

In [ ]:
# Create lists of features and target names
features = [col for col in df.columns
            if df[col].dtype=='float64' or col =='Type']
target = ['Target','Failure Type']
# Portion of data where RNF=1
idx_RNF = df.loc[df['Failure Type']=='Random Failures'].index
df.loc[idx_RNF,target]

In [ ]:
first_drop = df.loc[idx_RNF,target].shape[0]
print('Number of observations where RNF=1 but Machine failure=0:',first_drop)
# Drop corresponding observations and RNF column
df.drop(index=idx_RNF, inplace=True)

In [ ]:
# Portion of data where Machine failure=1 but no failure cause is specified
idx_ambiguous = df.loc[(df['Target']==1) &
                       (df['Failure Type']=='No Failure')].index
second_drop = df.loc[idx_ambiguous].shape[0]
print('Number of ambiguous observations:', second_drop)
display(df.loc[idx_ambiguous,target])
df.drop(index=idx_ambiguous, inplace=True)

In [ ]:
# Global percentage of removed observations
print('Global percentage of removed observations:',
     (100*(first_drop+second_drop)/n))
df.reset_index(drop=True, inplace=True)   # Reset index
n = df.shape[0]

In [ ]:
df.describe()

In [ ]:
num_features = [feature for feature in features if df[feature].dtype=='float64']

In [ ]:
# Portion of df where there is a failure and causes percentage
idx_fail = df.loc[df['Failure Type'] != 'No Failure'].index
df_fail = df.loc[idx_fail]
df_fail_percentage = 100*df_fail['Failure Type'].value_counts()/df_fail['Failure Type'].shape[0]
print('Failures percentage in data:',
      round(100*df['Target'].sum()/n,2))

In [ ]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTENC
# n_working must represent 80% of the desired length of resampled dataframe
n_working = df['Failure Type'].value_counts()['No Failure']
desired_length = round(n_working/0.8)
spc = round((desired_length-n_working)/4)  #samples per class
# Resampling
balance_cause = {'No Failure':n_working,
                 'Overstrain Failure':spc,
                 'Heat Dissipation Failure':spc,
                 'Power Failure':spc,
                 'Tool Wear Failure':spc}
sm = SMOTENC(categorical_features=[0,7], sampling_strategy=balance_cause, random_state=0)
df_res, y_res = sm.fit_resample(df, df['Failure Type'])

In [ ]:
# Portion of df_res where there is a failure and causes percentage
idx_fail_res = df_res.loc[df_res['Failure Type'] != 'No Failure'].index
df_res_fail = df_res.loc[idx_fail_res]
fail_res_percentage = 100*df_res_fail['Failure Type'].value_counts()/df_res_fail.shape[0]

# Percentages
print('Percentage increment of observations after oversampling:',
      round((df_res.shape[0]-df.shape[0])*100/df.shape[0],2))
print('SMOTE Resampled Failures percentage:',
      round(df_res_fail.shape[0]*100/df_res.shape[0],2))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

sc = StandardScaler()
type_dict = {'L': 0, 'M': 1, 'H': 2}
cause_dict = {'No Failure': 0,
              'Power Failure': 1,
              'Overstrain Failure': 2,
              'Heat Dissipation Failure': 3,
              'Tool Wear Failure': 4}
df_pre = df_res.copy()
# Encoding
df_pre['Type'].replace(to_replace=type_dict, inplace=True)
df_pre['Failure Type'].replace(to_replace=cause_dict, inplace=True)
# Scaling
df_pre[num_features] = sc.fit_transform(df_pre[num_features])

In [ ]:
pca = PCA(n_components=len(num_features))
X_pca = pd.DataFrame(data=pca.fit_transform(df_pre[num_features]), columns=['PC'+str(i+1) for i in range(len(num_features))])
var_exp = pd.Series(data=100*pca.explained_variance_ratio_, index=['PC'+str(i+1) for i in range(len(num_features))])
print('Explained variance ratio per component:', round(var_exp,2), sep='\n')
print('Explained variance ratio with 3 components: '+str(round(var_exp.values[:3].sum(),2)))

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score, fbeta_score
from sklearn.metrics import confusion_matrix, make_scorer
from sklearn.inspection import permutation_importance
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import time

# train-validation-test split
X, y = df_pre[features], df_pre[['Target','Failure Type']]
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.1, stratify=df_pre['Failure Type'], random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.11, stratify=y_trainval['Failure Type'], random_state=0)

In [69]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train['Failure Type'].values, dtype=torch.long)

X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val['Failure Type'].values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test['Failure Type'].values, dtype=torch.long)

# Create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Create DataLoader for test
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Create DataLoader for validation
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define the neural network architecture
class MLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLPClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


def evaluate(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    return accuracy

# Initialize the model
input_size = X_train.shape[1]
hidden_size = 128  # Adjust as needed
output_size = len(df_res['Failure Type'].unique())  # Number of classes
model = MLPClassifier(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

import torch

def calculate_loss(model, data_loader, criterion):
    model.eval()
    total_loss = 0.0
    num_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * len(labels)
            num_samples += len(labels)

    return total_loss / num_samples

def calculate_accuracy(model, data_loader):
    model.eval()
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():
        for inputs, labels in data_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += len(labels)

    return correct_predictions / total_samples


In [63]:
# Assuming you have initialized empty lists before the training loop
train_loss_list = []
val_loss_list = []
train_accuracy_list = []
val_accuracy_list = []

# Create a function to train and evaluate the model
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs):
    for epoch in range(num_epochs):
        # Training loop
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Calculate training loss and accuracy
        train_loss = calculate_loss(model, train_loader, criterion)
        train_accuracy = calculate_accuracy(model, train_loader)

        # Validation loop
        model.eval()
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        # Calculate validation loss and accuracy
        val_loss = calculate_loss(model, val_loader, criterion)
        val_accuracy = calculate_accuracy(model, val_loader)

        # Store metrics
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        train_accuracy_list.append(train_accuracy)
        val_accuracy_list.append(val_accuracy)

        # Print validation accuracy
        print(f'Epoch {epoch + 1}/{num_epochs}, Validation Accuracy: {val_accuracy:.4f}, Validation Loss: {val_loss:.4f}')

    return model, train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list

# Set up hyperparameter grid
param_grid = {
    'hidden_size': [16,32, 64, 128],
    'lr': [0.001, 0.01, 0.1],
    'optimizer': ['adam', 'sgd']
}

# Grid search
best_params = None
best_val_accuracy = 0.0

for hidden_size in param_grid['hidden_size']:
    for lr in param_grid['lr']:
        for optimizer_type in param_grid['optimizer']:
    
            # Initialize the model with current hyperparameters
            model = MLPClassifier(input_size, hidden_size, output_size)

            if optimizer_type == 'adam':
                optimizer = optim.Adam(model.parameters(), lr=lr)
            elif optimizer_type == 'sgd':
                optimizer = optim.SGD(model.parameters(), lr=lr)
            else:
                raise ValueError(f"Unsupported optimizer: {optimizer_type}")

            # Train and evaluate the model
            train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

            # Check validation accuracy
            val_accuracy = evaluate(model, val_loader)
            print(f'Hidden Size: {hidden_size}, Learning Rate: {lr}, Optimizer: {optimizer_type}, Validation Accuracy: {val_accuracy}')

            # Update best parameters if needed
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                best_params = {'hidden_size': hidden_size, 'lr': lr, 'Optimizer': optimizer_type}

print('Best Hyperparameters:', best_params)

Epoch 1/10, Validation Accuracy: 0.7998, Validation Loss: 0.7090
Epoch 2/10, Validation Accuracy: 0.8116, Validation Loss: 0.5472
Epoch 3/10, Validation Accuracy: 0.8459, Validation Loss: 0.4550
Epoch 4/10, Validation Accuracy: 0.8693, Validation Loss: 0.3824
Epoch 5/10, Validation Accuracy: 0.8844, Validation Loss: 0.3300
Epoch 6/10, Validation Accuracy: 0.8953, Validation Loss: 0.2941
Epoch 7/10, Validation Accuracy: 0.8995, Validation Loss: 0.2681
Epoch 8/10, Validation Accuracy: 0.9112, Validation Loss: 0.2477
Epoch 9/10, Validation Accuracy: 0.9087, Validation Loss: 0.2329
Epoch 10/10, Validation Accuracy: 0.9171, Validation Loss: 0.2205
Hidden Size: 16, Learning Rate: 0.001, Optimizer: adam, Validation Accuracy: 0.9170854271356784
Epoch 1/10, Validation Accuracy: 0.3769, Validation Loss: 1.5276
Epoch 2/10, Validation Accuracy: 0.5494, Validation Loss: 1.4059
Epoch 3/10, Validation Accuracy: 0.6910, Validation Loss: 1.3017
Epoch 4/10, Validation Accuracy: 0.7747, Validation Loss: 

In [67]:
import plotly.graph_objects as go

def plot_metrics(fig, title, x_label, y_label, train_data, val_data, mode='lines'):
    fig.add_trace(go.Scatter(x=list(range(1, len(train_data) + 1)),
                             y=train_data,
                             mode=mode,
                             name='Train'))
    fig.add_trace(go.Scatter(x=list(range(1, len(val_data) + 1)),
                             y=val_data,
                             mode=mode,
                             name='Validation'))
    fig.update_layout(title=title,
                      xaxis=dict(title=x_label),
                      yaxis=dict(title=y_label))

# Create the figure for Loss
fig_loss = go.Figure()
plot_metrics(fig_loss, 'Model Loss Over Epochs', 'Epoch', 'Loss', train_loss_list, val_loss_list)
fig_loss.show()

# Create the figure for Accuracy
fig_accuracy = go.Figure()
plot_metrics(fig_accuracy, 'Model Accuracy Over Epochs', 'Epoch', 'Accuracy', train_accuracy_list, val_accuracy_list)
fig_accuracy.show()

In [70]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def evaluate_metrics(model, loader, criterion):
    model.eval()
    loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            loss += criterion(outputs, labels).item()

            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    confusion_mat = confusion_matrix(all_labels, all_preds)
    classification_rep = classification_report(all_labels, all_preds)

    return avg_loss, accuracy, confusion_mat, classification_rep

# Assuming you have model, train_loader, and test_loader
train_loss, train_accuracy, train_conf_mat, train_classification_rep = evaluate_metrics(model, train_loader, criterion)
test_loss, test_accuracy, test_conf_mat, test_classification_rep = evaluate_metrics(model, test_loader, criterion)

print("Training Metrics:")
print(f"Loss: {train_loss:.4f}")
print(f"Accuracy: {train_accuracy:.4f}")
print(f"Confusion Matrix:\n{train_conf_mat}")
print(f"Classification Report:\n{train_classification_rep}")

print("\nTesting Metrics:")
print(f"Loss: {test_loss:.4f}")
print(f"Accuracy: {test_accuracy:.4f}")
print(f"Confusion Matrix:\n{test_conf_mat}")
print(f"Classification Report:\n{test_classification_rep}")


Training Metrics:
Loss: 1.5953
Accuracy: 0.3395
Confusion Matrix:
[[2981   11 4510    3  218]
 [ 164    1  170    0  148]
 [  92    0  293    0   98]
 [ 135    0  348    0    0]
 [ 378    0  102    0    3]]
Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.39      0.52      7723
           1       0.08      0.00      0.00       483
           2       0.05      0.61      0.10       483
           3       0.00      0.00      0.00       483
           4       0.01      0.01      0.01       483

    accuracy                           0.34      9655
   macro avg       0.19      0.20      0.13      9655
weighted avg       0.64      0.34      0.42      9655


Testing Metrics:
Loss: 1.5829
Accuracy: 0.3557
Confusion Matrix:
[[392   1 547   0  25]
 [ 19   0  26   0  16]
 [ 16   0  36   0   8]
 [ 19   0  41   0   0]
 [ 50   0   9   0   1]]
Classification Report:
              precision    recall  f1-score   support

           0       0.

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [71]:
import torch.nn as nn

class RegularizedModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(RegularizedModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout_rate)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Usage
regularized_model = RegularizedModel(input_size, hidden_size, output_size)


In [72]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Define model with regularization techniques
class RegularizedModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate=0.5):
        super(RegularizedModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.dropout = nn.Dropout(p=dropout_rate)
        self.batch_norm = nn.BatchNorm1d(hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.batch_norm(x)
        x = self.fc2(x)
        return x

# Function to train and evaluate the model
def train_and_evaluate(model, train_loader, val_loader, criterion, optimizer, num_epochs=10, patience=3):
    train_loss_list, val_loss_list = [], []
    train_accuracy_list, val_accuracy_list = [], []
    best_val_loss = float('inf')
    early_stopping_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss, train_accuracy = 0.0, 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            train_accuracy += accuracy_score(labels.cpu(), preds.cpu())

        # Calculate average training loss and accuracy
        train_loss = train_loss / len(train_loader.dataset)
        train_accuracy = train_accuracy / len(train_loader)

        # Evaluate on the validation set
        model.eval()
        val_loss, val_accuracy = 0.0, 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * inputs.size(0)
                _, preds = torch.max(outputs, 1)
                val_accuracy += accuracy_score(labels.cpu(), preds.cpu())

        # Calculate average validation loss and accuracy
        val_loss = val_loss / len(val_loader.dataset)
        val_accuracy = val_accuracy / len(val_loader)

        # Save metrics for plotting
        train_loss_list.append(train_loss)
        val_loss_list.append(val_loss)
        train_accuracy_list.append(train_accuracy)
        val_accuracy_list.append(val_accuracy)

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stopping_counter = 0
        else:
            early_stopping_counter += 1
            if early_stopping_counter > patience:
                print("Early stopping. Training stopped.")
                break

        # Print progress
        print(f"Epoch {epoch + 1}/{num_epochs}: "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f}, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.4f}")

    return train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list

# Function to plot metrics
def plot_metrics(train_metric, val_metric, metric_name):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=list(range(1, len(train_metric) + 1)), y=train_metric, mode='lines', name='Train'))
    fig.add_trace(go.Scatter(x=list(range(1, len(val_metric) + 1)), y=val_metric, mode='lines', name='Validation'))
    fig.update_layout(title=f'Model {metric_name} Over Epochs',
                      xaxis=dict(title='Epoch'),
                      yaxis=dict(title=metric_name))
    fig.show()

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train['Failure Type'].values, dtype=torch.long)

X_val_tensor = torch.tensor(X_val.values, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val['Failure Type'].values, dtype=torch.long)

# Create DataLoader for training
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Create DataLoader for validation
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define model, optimizer, and criterion
model = RegularizedModel(input_size=X_train.shape[1], hidden_size=64, output_size=len(set(y_train['Failure Type'])))
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

# Train and evaluate the model
train_loss_list, val_loss_list, train_accuracy_list, val_accuracy_list = train_and_evaluate(
    model, train_loader, val_loader, criterion, optimizer, num_epochs=100, patience=5
)

# Plot loss and accuracy curves
plot_metrics(train_loss_list, val_loss_list, 'Loss')
plot_metrics(train_accuracy_list, val_accuracy_list, 'Accuracy')


Epoch 1/100: Train Loss: 1.1180, Train Acc: 0.6601, Val Loss: 0.5966, Val Acc: 0.8773
Epoch 2/100: Train Loss: 0.5241, Train Acc: 0.8465, Val Loss: 0.3409, Val Acc: 0.8958
Epoch 3/100: Train Loss: 0.3847, Train Acc: 0.8721, Val Loss: 0.2732, Val Acc: 0.9057
Epoch 4/100: Train Loss: 0.3353, Train Acc: 0.8827, Val Loss: 0.2326, Val Acc: 0.9115
Epoch 5/100: Train Loss: 0.2992, Train Acc: 0.8926, Val Loss: 0.2095, Val Acc: 0.9221
Epoch 6/100: Train Loss: 0.2884, Train Acc: 0.8936, Val Loss: 0.2042, Val Acc: 0.9156
Epoch 7/100: Train Loss: 0.2843, Train Acc: 0.8972, Val Loss: 0.1892, Val Acc: 0.9296
Epoch 8/100: Train Loss: 0.2731, Train Acc: 0.8959, Val Loss: 0.1868, Val Acc: 0.9234
Epoch 9/100: Train Loss: 0.2676, Train Acc: 0.8963, Val Loss: 0.1834, Val Acc: 0.9242
Epoch 10/100: Train Loss: 0.2493, Train Acc: 0.9066, Val Loss: 0.1772, Val Acc: 0.9292
Epoch 11/100: Train Loss: 0.2441, Train Acc: 0.9059, Val Loss: 0.1729, Val Acc: 0.9250
Epoch 12/100: Train Loss: 0.2425, Train Acc: 0.9054,